# Bibliotecas

In [ ]:
import numpy    as np
import pandas   as pd
import xgboost  as xgb

from typing                     import Literal
from sklearn.ensemble           import RandomForestRegressor
from sklearn.linear_model       import LinearRegression, Lasso
from sklearn.dummy              import DummyRegressor
from sklearn.neural_network     import MLPRegressor
from sklearn.metrics            import mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error, r2_score

# Importação de dados

In [2]:
X_train = pd.read_pickle('../exports/cicle_exports/06_feature_selection/X_train.pkl', compression='gzip')
y_train = pd.read_pickle('../exports/cicle_exports/06_feature_selection/y_train.pkl', compression='gzip')

X_val = pd.read_pickle('../exports/cicle_exports/06_feature_selection/X_val.pkl', compression='gzip')
y_val = pd.read_pickle('../exports/cicle_exports/06_feature_selection/y_val.pkl', compression='gzip')

In [3]:
# Dados para utilizar no cross validation:
X_cv = pd.concat([X_train, X_val], ignore_index=True)
y_cv = pd.concat([y_train, y_val], ignore_index=True)

In [4]:
# Convertendo para ndarray para treinamentos fora do cross validation:
X_train_np = X_train.drop(['date'], axis=1).values
y_train_np = y_train.values.ravel()

X_val_np = X_val.drop(['date'], axis=1).values
y_val_np = y_val.values.ravel()

# Funções

In [5]:
def ml_error_regression( model_name:str, y_pred:np.ndarray, y:np.ndarray ):
    """
    Gera métricas de performance de um modelo de regressão.

    Parâmetros:
        model_name (str): Nome do modelo para exibição.
        y_pred (np.ndarray): Previsões da variável alvo geradas pelo modelo.
        y (np.ndarray): Variável alvo do dataset original.

    Retorna:
        pd.DataFrame: DataFrame com métricas (mae, mape, rmse).
    """
    def safe_metric(func, y_true, y_pred):
            try:
                value = func(y_true, y_pred)
                if np.isinf(value) or np.isnan(value):
                    return 'inf'
                return np.round(value, 2).astype(str)
            except (ValueError, FloatingPointError, OverflowError, ZeroDivisionError):
                return 'inf'

    score_mae     = safe_metric(mean_absolute_error, y, y_pred)
    score_mape    = safe_metric(mean_absolute_percentage_error, y, y_pred)
    score_rmse    = safe_metric(root_mean_squared_error, y, y_pred)
    score_r2      = safe_metric(r2_score, y, y_pred)

    return pd.DataFrame( {'Model Name'  : model_name, 
                          'MAE'         : score_mae, 
                          'MAPE'        : score_mape, 
                          'RMSE'        : score_rmse,
                          'R2'          : score_r2
                          }, index=[0] )

In [6]:
def ml_cross_validation_regression_timeseries_by_day(X: pd.DataFrame, 
                                                      y: pd.Series, model, 
                                                      model_name: str, 
                                                      k_folds: int, 
                                                      model_pack: Literal['scikit-learn', 'tensorflow_lstm'] = 'scikit-learn'
                                                      ) -> None:
    """
    Validação cruzada para regressão em séries temporais com base em ano-mês-dia.

    Parâmetros:
        X (pd.DataFrame): DataFrame de features, deve conter coluna 'date' do tipo datetime.
        y (pd.Series): Série com os valores alvo.
        model: Modelo de regressão (Scikit-learn).
        model_name (str): Nome do modelo.
        k_folds (int): Número de folds (divisão sequencial por ano-semana).

    Retorna:
        pd.DataFrame com MAE, MAPE e RMSE médios e desvios padrão.
    """

    # Combina X e y temporariamente para facilitar a filtragem por semana
    df = X.copy()
    df['target'] = y.values

    if 'date' not in df.columns:
        raise ValueError("O DataFrame X deve conter a coluna 'date' com timestamps.")

    df['year_month_day'] = df['date'].dt.strftime('%Y-%m-%d')
    unique_days = sorted(df['year_month_day'].unique())
    folds = np.array_split(unique_days, k_folds)

    scores_mae  = []
    scores_mape = []
    scores_rmse = []
    scores_r2   = []

    for i in range(1, k_folds):
        train_days = np.concatenate(folds[:i])
        test_days = folds[i]

        train_data = df[df['year_month_day'].isin(train_days)].copy()
        test_data  = df[df['year_month_day'].isin(test_days)].copy()

        # Remover colunas temporais antes do treino
        drop_cols = ['date', 'year_month_day', 'target']
        X_train_k = train_data.drop(columns=drop_cols).values
        X_test_k  = test_data.drop(columns=drop_cols).values
        y_train_k = train_data['target'].values.ravel()
        y_test_k  = test_data['target'].values.ravel()

        if model_pack == 'scikit-learn':
            model.fit(X_train_k, y_train_k)
        elif model_pack == 'tensorflow_lstm':
            model.fit(X_train_k, y_train_k, epochs=20, batch_size=16)
            
        y_pred_k = model.predict(X_test_k)

        scores_mae  .append(mean_absolute_error(np.expm1(y_test_k), np.expm1(y_pred_k)))
        scores_mape .append(mean_absolute_percentage_error(np.expm1(y_test_k), np.expm1(y_pred_k)))
        scores_rmse .append(root_mean_squared_error(np.expm1(y_test_k), np.expm1(y_pred_k)))
        scores_r2   .append(r2_score(np.expm1(y_test_k), np.expm1(y_pred_k)))
        # np.expm1, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis

    return pd.DataFrame({
                        'Model Name': model_name,
                        'MAE'       : f"{np.round(np.mean(scores_mae), 2)} +/- {np.round(np.std(scores_mae), 2)}",
                        'MAPE'      : f"{np.round(np.mean(scores_mape), 2)} +/- {np.round(np.std(scores_mape), 2)}",
                        'RMSE'      : f"{np.round(np.mean(scores_rmse), 2)} +/- {np.round(np.std(scores_rmse), 2)}",
                        'R2'        : f"{np.round(np.mean(scores_r2), 2)} +/- {np.round(np.std(scores_r2), 2)}"
                        }, index=[0])


# 7.0. Modelos de machine learning

## 7.1. Modelos testados

### 7.1.1. Modelo de média (Referência)

In [7]:
# Definição do modelo:
model_random = DummyRegressor(strategy='mean')

# Treinamento:
model_random.fit(X_train_np, y_train_np)

# Previsão:
y_pred_random = model_random.predict(X_val_np)

In [8]:
# Avaliação simples:
result_random = ml_error_regression('Média Simples (Referência)', 
                                    np.expm1(y_pred_random),    # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis
                                    np.expm1(y_val_np)          # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis
                                    ) 
result_random

,Model Name,MAE,MAPE,RMSE,R2
0,Média Simples (Referência),2354.69,0.34,3371.29,-0.11


In [9]:
# Cross validation:
cv_random = ml_cross_validation_regression_timeseries_by_day(X_cv, y_cv, model_random, 'Média Simples (Referência)', 5)
cv_random

,Model Name,MAE,MAPE,RMSE,R2
0,Média Simples (Referência),2252.6 +/- 114.53,0.35 +/- 0.01,3213.82 +/- 203.55,-0.07 +/- 0.03


### 7.1.2. Linear Regression Model

In [10]:
# Definição do modelo:
model_lr = LinearRegression(n_jobs=-1)

# Treinamento:
model_lr.fit(X_train_np, y_train_np)

# Previsão:
y_pred_lr = model_lr.predict(X_val_np)

In [11]:
# Avaliação simples:
result_lr = ml_error_regression('Linear Regression', 
                                np.expm1(y_pred_lr), # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis
                                np.expm1(y_val_np)   # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis
                                )
result_lr

,Model Name,MAE,MAPE,RMSE,R2
0,Linear Regression,2087.87,0.3,3020.51,0.11


In [12]:
# Cross validation:
cv_lr = ml_cross_validation_regression_timeseries_by_day(X_cv, y_cv, model_lr, 'Linear Regression', 5)
cv_lr

,Model Name,MAE,MAPE,RMSE,R2
0,Linear Regression,2006.02 +/- 99.19,0.3 +/- 0.01,2926.33 +/- 168.61,0.11 +/- 0.04


### 7.1.3. Linear Regression Regularized Model - Lasso

In [13]:
# Definição do modelo:
model_lasso = Lasso(alpha=0.01)

# Treinamento:
model_lasso.fit(X_train_np, y_train_np)

# Previsão:
y_pred_lasso = model_lasso.predict(X_val_np)

In [14]:
# Avaliação simples:
result_lasso = ml_error_regression('Linear Regression - Lasso', 
                                   np.expm1(y_pred_lasso),  # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis
                                   np.expm1(y_val_np)       # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis
                                   )
result_lasso

,Model Name,MAE,MAPE,RMSE,R2
0,Linear Regression - Lasso,2115.76,0.3,3085.4,0.07


In [15]:
# Cross validation:
cv_lasso = ml_cross_validation_regression_timeseries_by_day(X_cv, y_cv, model_lasso, 'Linear Regression - Lasso', 5)
cv_lasso

,Model Name,MAE,MAPE,RMSE,R2
0,Linear Regression - Lasso,2010.67 +/- 112.79,0.3 +/- 0.0,2931.49 +/- 202.26,0.11 +/- 0.04


### 7.1.4. Random Forest Regressor

In [16]:
# Definição do modelo:
model_rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)

# Treinamento:
model_rf.fit(X_train_np, y_train_np)

# Previsão:
y_pred_rf = model_rf.predict(X_val_np)

In [17]:
# Avaliação simples:
result_rf = ml_error_regression('Random Forest Regressor', 
                                np.expm1(y_pred_rf),    # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis 
                                np.expm1(y_val_np)      # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis
                                )
result_rf

,Model Name,MAE,MAPE,RMSE,R2
0,Random Forest Regressor,961.32,0.14,1518.37,0.78


In [18]:
# Cross validation:
cv_rf = ml_cross_validation_regression_timeseries_by_day(X_cv, y_cv, model_rf, 'Random Forest Regressor', 5)
cv_rf

,Model Name,MAE,MAPE,RMSE,R2
0,Random Forest Regressor,963.35 +/- 117.62,0.14 +/- 0.01,1502.34 +/- 225.77,0.76 +/- 0.05


### 7.1.5. XGBoost

In [19]:
# Definição do modelo:
model_xgb = xgb.XGBRegressor( objective='reg:squarederror',
                              n_estimators=100, 
                              eta=0.01, 
                              max_depth=10, 
                              subsample=0.7,
                            )

# Treinamento:
model_xgb.fit(X_train_np, y_train_np)

# Previsão:
y_pred_xgb = model_xgb.predict(X_val_np)

In [20]:
# Avaliação simples:
result_xgb = ml_error_regression('XGBoost', 
                                 np.expm1(y_pred_xgb),   # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis 
                                 np.expm1(y_val_np)      # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis
                                 )
result_xgb

,Model Name,MAE,MAPE,RMSE,R2
0,XGBoost,1862.3,0.26,2706.75,0.29


In [21]:
# Cross validation:
cv_xgb = ml_cross_validation_regression_timeseries_by_day(X_cv, y_cv, model_xgb, 'XGBoost', 5)
cv_xgb

,Model Name,MAE,MAPE,RMSE,R2
0,XGBoost,1788.22 +/- 111.58,0.26 +/- 0.0,2611.78 +/- 199.73,0.29 +/- 0.04


### 7.1.6. MLPRegressor

In [22]:
# Definição do modelo:
model_mlp = MLPRegressor(max_iter=1000)

# Treinamento:
model_mlp.fit(X_train_np, y_train_np)

# Previsão:
y_pred_mlp = model_mlp.predict(X_val_np)

In [23]:
# Avaliação simples:
result_mlp = ml_error_regression('MLPRegressor', 
                                 np.expm1(y_pred_mlp),   # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis 
                                 np.expm1(y_val_np)      # exp, pois a variável tinha sido transformada para escala logarítmica para seleção de variáveis
                                 )
result_mlp

,Model Name,MAE,MAPE,RMSE,R2
0,MLPRegressor,1876.67,0.25,2772.36,0.25


In [25]:
# Cross validation:
cv_mlp = ml_cross_validation_regression_timeseries_by_day(X_cv, y_cv, model_mlp, 'MLPRegressor', 5)
cv_mlp

,Model Name,MAE,MAPE,RMSE,R2
0,MLPRegressor,2563.26 +/- 1148.02,0.37 +/- 0.17,3437.49 +/- 1152.86,-0.46 +/- 1.21


## 7.2. Comparação dos modelos

### 7.2.1. Performance simples

In [28]:
modelling_result = pd.concat( [result_random, result_lr, result_lasso, result_rf, result_xgb, result_mlp] )

modelling_result = modelling_result.sort_values( 'RMSE', ascending=True )
modelling_result

,Model Name,MAE,MAPE,RMSE,R2
0,Random Forest Regressor,961.32,0.14,1518.37,0.78
0,XGBoost,1862.3,0.26,2706.75,0.29
0,MLPRegressor,1876.67,0.25,2772.36,0.25
0,Linear Regression,2087.87,0.3,3020.51,0.11
0,Linear Regression - Lasso,2115.76,0.3,3085.4,0.07
0,Média Simples (Referência),2354.69,0.34,3371.29,-0.11


### 7.2.2. Performance real - Cross validation

In [29]:
modelling_result_cv = pd.concat( [cv_random, cv_lr, cv_lasso, cv_rf, cv_xgb, cv_mlp] )
modelling_result_cv = modelling_result_cv.sort_values( 'RMSE', ascending=True )
modelling_result_cv

,Model Name,MAE,MAPE,RMSE,R2
0,Random Forest Regressor,963.35 +/- 117.62,0.14 +/- 0.01,1502.34 +/- 225.77,0.76 +/- 0.05
0,XGBoost,1788.22 +/- 111.58,0.26 +/- 0.0,2611.78 +/- 199.73,0.29 +/- 0.04
0,Linear Regression,2006.02 +/- 99.19,0.3 +/- 0.01,2926.33 +/- 168.61,0.11 +/- 0.04
0,Linear Regression - Lasso,2010.67 +/- 112.79,0.3 +/- 0.0,2931.49 +/- 202.26,0.11 +/- 0.04
0,Média Simples (Referência),2252.6 +/- 114.53,0.35 +/- 0.01,3213.82 +/- 203.55,-0.07 +/- 0.03
0,MLPRegressor,2563.26 +/- 1148.02,0.37 +/- 0.17,3437.49 +/- 1152.86,-0.46 +/- 1.21


Modelo escolhido: Embora o **Random Forest Regressor** tenha obtido um desempenho muito superior aos demais, é um algoritmo muito pesado computacionalmente e também em tamanho de arquivo. Portanto será realizada a otimização do **XGBoost** (segundo colocado na lista ordenada por menor RMSE) e se após esse processo o algoritmo apresentar um resultado satisfatório, será o algoritmo definitivo.

# Exportação de resultados

In [31]:
modelling_result.to_pickle      ('../exports/cicle_exports/07_machine_learning/modelling_result.pkl', compression='gzip')
modelling_result_cv.to_pickle   ('../exports/cicle_exports/07_machine_learning/modelling_result_cv.pkl', compression='gzip')